<a href="https://colab.research.google.com/github/SaashaJoshi/generating-handwritten-digits-GAN/blob/master/handwritten_digits_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [0]:
from keras.datasets import mnist
(train_imgs, train_labels), (test_imgs, test_labels)=mnist.load_data()

In [0]:
print('Train: ', train_imgs.shape, train_labels.shape)
print('Test: ', test_imgs.shape, test_labels.shape)

In [0]:
plt.imshow(test_imgs[0], cmap='gray')

In [0]:
plt.imshow(test_imgs[10], cmap='gray_r')

In [0]:
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.axis('off')
  plt.imshow(train_imgs[i], cmap='gray_r')
  
plt.show()

In [0]:
# DISCRIMINATOR MODEL

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
 
def define_discriminator(in_shape=(28,28,1)):
  model=Sequential()
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

'''
model=keras.Sequential([
    keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=(28, 28, 1)), 
    keras.layers.LeakyReLU(alpha=0.2), 
    keras.layers.Dropout(0.4), 
    keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'), 
    keras.layers.LeakyReLU(alpha=0.2), 
    keras.layers.Dropout(0.4), 
    keras.layers.Flatten(), 
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', 
             optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5), 
             metrics=['accuracy'])

model.summary()
keras.utils.plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)
'''

In [0]:
# Load MNIST training images
def load_real_samples():
  (train_imgs, _), (_, _)=mnist.load_data()
  train_img_samples=np.expand_dims(train_imgs, axis=-1)
  train_img_samples=train_img_samples.astype('float32')/255
  return train_img_samples

# Select random real samples
def generate_real_samples(dataset, n_samples):
  x=np.random.randint(0, dataset.shape[0], n_samples)
  train_img_samples=dataset[x]
  train_label_samples=np.ones((n_samples, 1))
  return train_img_samples, train_label_samples

In [0]:
# Select random fake samples
def generate_fake_samples(n_samples):
  train_img_samples=np.random.rand(28*28*n_samples)
  train_img_samples=train_img_samples.reshape((n_samples, 28, 28, 1))
  train_label_samples=np.zeros((n_samples, 1))
  return train_img_samples, train_label_samples

In [0]:
# TRAIN DISCRIMINATOR MODEL
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
  half_batch=int(n_batch/2)
  
  for i in range(n_iter):
    img_real, label_real=generate_real_samples(dataset, half_batch)
    _, real_accuracy=model.train_on_batch(img_real, label_real)
    
    img_fake, label_fake=generate_fake_samples(half_batch)
    _, fake_accuracy=model.train_on_batch(img_fake, label_fake)
    
    print('{} Real: {}% Fake: {}%'.format(i+1, real_accuracy*100, fake_accuracy*100))

In [0]:
model=define_discriminator()
dataset=load_real_samples()
train_discriminator(model, dataset)

In [0]:
model.summary()
keras.utils.plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
# GENERATOR MODEL

def define_generator(latent_dim):
  model=Sequential()
  n_nodes=128*7*7
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((7, 7, 128)))
  model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(1, (7, 7), activation='sigmoid', padding='same'))
  return model

In [0]:
latent_dim=100   #100 element vector; can be 10, 50, 500 etc.
model=define_generator(latent_dim)
model.summary()
keras.utils.plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
def generate_latent_points(latent_dim, n_samples):
  latent_points=np.random.randn(latent_dim*n_samples)
  latent_points=latent_points.reshape(n_samples, latent_dim)
  return latent_points

In [0]:
def generate_fake_samples(g_model, latent_dim, n_samples):
  latent_points=generate_latent_points(latent_dim, n_samples)
  img_fake_samples=g_model.predict(latent_points)
  label_fake_samples=np.zeros((n_samples, 1))
  return img_fake_samples, label_fake_samples

In [0]:
n_samples=25
img_fake_samples, _=generate_fake_samples(model, latent_dim, n_samples)

for i in range(n_samples):
  plt.subplot(5, 5, i+1)
  plt.axis('off')
  plt.imshow(img_fake_samples[i, :, :, 0], cmap='gray_r')
  
plt.show()